# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [50]:
from pyspark.sql import SparkSession

In [51]:
spark = SparkSession.builder.appName('customer').getOrCreate()

In [52]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [53]:
df.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [72]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [73]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [74]:
City=df.select("Location")
City.show(5)

+--------------------+
|            Location|
+--------------------+
|10265 Elizabeth M...|
|6157 Frank Garden...|
|1331 Keith Court ...|
|13120 Daniel Moun...|
|765 Tricia Row Ka...|
+--------------------+
only showing top 5 rows



In [93]:
Assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [94]:
AsmData = Assembler.transform(df)

In [98]:
Final_Data = AsmData.select("features", "Churn")

In [99]:
from pyspark.ml.classification import LogisticRegression

In [100]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Churn')

In [103]:
train, test = Final_Data.randomSplit([0.7,0.3])

In [104]:
fit_model = log_reg.fit(train)

In [105]:
results = fit_model.transform(train)

In [106]:
train_results = results.summary()

In [107]:
train_results.show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                624|                624|
|   mean|0.18108974358974358|0.14102564102564102|
| stddev| 0.3854014583106712|0.34832721924783633|
|    min|                  0|                0.0|
|    25%|                  0|                0.0|
|    50%|                  0|                0.0|
|    75%|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



In [108]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [109]:
pred_labels = fit_model.evaluate(test)

In [110]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.34393530601480...|[0.99824571013004...|       0.0|
|[28.0,9090.43,1.0...|    0|[0.97449453683978...|[0.72601444729530...|       0.0|
|[28.0,11204.23,0....|    0|[1.31030516783804...|[0.78756421701883...|       0.0|
|[28.0,11245.38,0....|    0|[3.76149283815506...|[0.97727922794807...|       0.0|
|[29.0,5900.78,1.0...|    0|[3.77687969881776...|[0.97761838865937...|       0.0|
|[30.0,8874.83,0.0...|    0|[2.99983887891801...|[0.95256684737934...|       0.0|
|[30.0,10183.98,1....|    0|[2.49565028620533...|[0.92383632575257...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.19505635806438...|[0.96064781479327...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.13733698220683...|[0.98428557483137...|       0.0|
|[32.0,6367.22,1

In [111]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Churn")

In [112]:
auc = churn_eval.evaluate(pred_labels.predictions)

In [113]:
auc

0.795035621395454

In [115]:
NewDF = df = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [116]:
NewDF.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [129]:
NewData = Assembler.transform(NewDF)

In [130]:
NewData.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [131]:
Predictions = fit_model.transform(NewData)

In [132]:
Predictions.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.24282392818771...|[0.90402974190111...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-7.2223616790360...|[7.29543553300465...|       

In [134]:
Predictions.select("Company","prediction").show(10)

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

